In [47]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer, AerSimulator
from qiskit.quantum_info import Statevector, state_fidelity
from qiskit_aer.noise import NoiseModel, amplitude_damping_error, depolarizing_error, phase_damping_error, pauli_error, kraus_error
from qiskit_ibm_runtime import QiskitRuntimeService

import time
import tracemalloc
import csv


In [52]:
# Initialize QiskitRuntimeService
service = QiskitRuntimeService(channel="ibm_quantum", token='23060337522268b928a8cd131be02a74efb2f4540c088b6bb9737d72ef8ee1c90e185d1eb2718a3cad12599debecf55f1709629d6c1051b40ad4a56f783ecded')

# List of available backends
available_backends = ['ibm_brisbane',  'ibm_kyiv', 'ibm_sherbrooke']

# Find the least busy backend
least_busy_backend = None
min_pending_jobs = float('inf')
for backend_name in available_backends:
    backend = service.backend(name=backend_name)
    status = backend.status()
    if status.operational and status.pending_jobs < min_pending_jobs:
        least_busy_backend = backend
        min_pending_jobs = status.pending_jobs

if least_busy_backend is None:
    raise ValueError("No operational backend found.")
else:
    print(f"Selected backend: {least_busy_backend.name}")


Selected backend: ibm_brisbane


In [35]:

# Define Quantum Error Correction Code Circuits

def shor_code_circuit(include_measure=True):
    qc = QuantumCircuit(9)
    
    # Encode logical |0> state (bit-flip encoding)
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(0, 2)
    
    qc.cx(3, 4)
    qc.cx(3, 5)
    
    qc.cx(6, 7)
    qc.cx(6, 8)
    
    # Phase-flip encoding (using CNOTs and H gates)
    for i in [0, 3, 6]:
        qc.h(i)
        qc.cx(i, i+1)
        qc.cx(i, i+2)
        qc.h(i)
    
    # Bit-flip protection
    qc.cx(0, 3)
    qc.cx(0, 6)
    qc.cx(1, 4)
    qc.cx(1, 7)
    qc.cx(2, 5)
    qc.cx(2, 8)

    # Optionally include measurement
    if include_measure:
        qc.measure_all()
    
    return qc

def steane_code_circuit(include_measure=True):
    qc = QuantumCircuit(7)

    # Encode logical |0> state
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(0, 2)
    
    qc.cx(0, 3)
    qc.cx(1, 4)
    qc.cx(2, 5)
    
    qc.h(0)
    qc.h(1)
    qc.h(2)
    
    qc.cx(0, 6)

    # Optionally include measurement
    if include_measure:
        qc.measure_all()
    
    return qc

def surface_code_circuit(include_measure=True):
    qc = QuantumCircuit(13)

    # Simplified example of a surface code (not a full implementation)
    # Initialize logical |0> state
    qc.h([0, 1, 2])
    
    # Create stabilizers
    for i in range(1, 7):
        qc.cx(0, i)
        qc.cz(0, i+6)

    # Optionally include measurement
    if include_measure:
        qc.measure_all()
    
    return qc



In [36]:
noise_model = NoiseModel.from_backend(backend)


KeyboardInterrupt: 

In [ ]:



# Noise Models
def create_depolarizing_noise_model(noise_level):
    noise_model = NoiseModel()
    depol_error_1q = depolarizing_error(noise_level, 1)
    depol_error_2q = depolarizing_error(noise_level, 2)
    noise_model.add_all_qubit_quantum_error(depol_error_1q, ["u3", "u2", "u1"])
    noise_model.add_all_qubit_quantum_error(depol_error_2q, ["cx"])
    return noise_model

def create_amplitude_damping_noise_model(noise_level):
    noise_model = NoiseModel()
    amp_damp_error_1q = amplitude_damping_error(noise_level)
    noise_model.add_all_qubit_quantum_error(amp_damp_error_1q, ["u3"])
    return noise_model

def create_phase_damping_noise_model(noise_level):
    noise_model = NoiseModel()
    phase_damp_error_1q = phase_damping_error(noise_level)
    noise_model.add_all_qubit_quantum_error(phase_damp_error_1q, ["u1"])
    return noise_model

def create_bit_flip_noise_model(noise_level):
    noise_model = NoiseModel()
    bit_flip_error = pauli_error([('X', noise_level), ('I', 1 - noise_level)])
    noise_model.add_all_qubit_quantum_error(bit_flip_error, ["x"])
    return noise_model

def create_phase_flip_noise_model(noise_level):
    noise_model = NoiseModel()
    phase_flip_error = pauli_error([('Z', noise_level), ('I', 1 - noise_level)])
    noise_model.add_all_qubit_quantum_error(phase_flip_error, ["z"])
    return noise_model

def create_bit_phase_flip_noise_model(noise_level):
    noise_model = NoiseModel()
    bit_phase_flip_error = pauli_error([('Y', noise_level), ('I', 1 - noise_level)])
    noise_model.add_all_qubit_quantum_error(bit_phase_flip_error, ["y"])
    return noise_model

def create_kraus_error_model(noise_level):
    noise_model = NoiseModel()
    K0 = np.array([[1, 0], [0, np.sqrt(1 - noise_level)]])
    K1 = np.array([[0, np.sqrt(noise_level)], [0, 0]])
    kraus_ops = [K0, K1]
    error = kraus_error(kraus_ops)
    noise_model.add_all_qubit_quantum_error(error, ["h", "u3", "x", "z"])
    return noise_model

In [ ]:

from qiskit.transpiler import CouplingMap, PassManager
from qiskit.transpiler.passes import SetLayout, FullAncillaAllocation, EnlargeWithAncilla, ApplyLayout



# Ensure that backend properties are properly initialized
if backend.configuration().coupling_map is None or backend.configuration().basis_gates is None:
    raise ValueError("The backend's coupling map or basis gates are not properly initialized.")

# Define Quantum Error Correction Code Circuits
def shor_code_circuit(include_measure=True):
    qc = QuantumCircuit(9)
    
    # Encode logical |0> state (bit-flip encoding)
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(0, 2)
    
    qc.cx(3, 4)
    qc.cx(3, 5)
    
    qc.cx(6, 7)
    qc.cx(6, 8)
    
    # Phase-flip encoding (using CNOTs and H gates)
    for i in [0, 3, 6]:
        qc.h(i)
        qc.cx(i, i+1)
        qc.cx(i, i+2)
        qc.h(i)
    
    # Bit-flip protection
    qc.cx(0, 3)
    qc.cx(0, 6)
    qc.cx(1, 4)
    qc.cx(1, 7)
    qc.cx(2, 5)
    qc.cx(2, 8)

    # Optionally include measurement
    if include_measure:
        qc.measure_all()
    
    return qc

def steane_code_circuit(include_measure=True):
    qc = QuantumCircuit(7)

    # Encode logical |0> state
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(0, 2)
    
    qc.cx(0, 3)
    qc.cx(1, 4)
    qc.cx(2, 5)
    
    qc.h(0)
    qc.h(1)
    qc.h(2)
    
    qc.cx(0, 6)

    # Optionally include measurement
    if include_measure:
        qc.measure_all()
    
    return qc

def surface_code_circuit(include_measure=True):
    qc = QuantumCircuit(13)

    # Simplified example of a surface code (not a full implementation)
    # Initialize logical |0> state
    qc.h([0, 1, 2])
    
    # Create stabilizers
    for i in range(1, 7):
        qc.cx(0, i)
        qc.cz(0, i+6)

    # Optionally include measurement
    if include_measure:
        qc.measure_all()
    
    return qc

# Validate Circuit without noise
def validate_circuit_ideal(code_name):
    if code_name == 'shor':
        qc_circuit = shor_code_circuit()
    elif code_name == 'steane':
        qc_circuit = steane_code_circuit()
    elif code_name == 'surface':
        qc_circuit = surface_code_circuit()
    else:
        raise ValueError(f"Unknown code name: {code_name}")
    
    # Transpile the circuit for the backend with explicit layout and routing method
    try:
        qc_transpiled = transpile(qc_circuit, backend, optimization_level=0, routing_method='basic', initial_layout=None)
        # Run the circuit and retrieve the measurement result
        result = backend.run(qc_transpiled).result()
        counts = result.get_counts(qc_transpiled)
        print(f"Measurement counts for {code_name} code:\n", counts)
    except Exception as e:
        print(f"Error in transpiling or running the circuit for {code_name} code: {str(e)}")

# Validate each QEC circuit without noise
validate_circuit_ideal('shor')
validate_circuit_ideal('steane')
validate_circuit_ideal('surface')


/var/folders/f2/97psbk3s4nq1wknv9swc91740000gn/T/ipykernel_43956/3700718137.py:103: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/api/migration-guides/qiskit-runtime.
  result = backend.run(qc_transpiled).result()


Measurement counts for shor code:
 {'010011100': 9, '110001001': 7, '110010100': 11, '001011110': 9, '000110111': 3, '100111011': 5, '011110111': 6, '011111110': 8, '010000100': 6, '000101001': 5, '011001001': 5, '000011000': 9, '111101100': 8, '110101001': 6, '011100000': 4, '011101000': 10, '100010111': 6, '000010111': 7, '100101001': 5, '000000100': 7, '100111010': 6, '111010010': 6, '001111000': 7, '001110110': 8, '100110011': 7, '111000000': 7, '101000001': 6, '010111010': 2, '011100111': 5, '100010100': 9, '010011010': 8, '110101000': 11, '111111110': 2, '100010000': 8, '001011100': 5, '010010100': 14, '011011001': 15, '000101100': 12, '110110011': 10, '111101111': 5, '100001111': 3, '010000011': 9, '110011110': 2, '110000100': 5, '010001111': 10, '011000100': 5, '111100101': 9, '110010110': 1, '000110110': 10, '010010010': 6, '011111001': 8, '101100100': 5, '110110000': 7, '011011110': 2, '100101010': 16, '001100101': 2, '111100001': 6, '100000111': 6, '000100011': 6, '010010101

In [ ]:


def calculate_error_rate(ideal_counts, noisy_counts, shots):
    """Calculate the error rate based on measurement counts."""
    ideal_total = sum(ideal_counts.values())
    correct_counts = sum(noisy_counts.get(bitstring, 0) for bitstring in ideal_counts)
    error_rate = 1 - (correct_counts / shots)
    return error_rate

def simulate_with_qec(noise_model, backend, code_name, shots=1024):
    if code_name == 'shor':
        qc_circuit = shor_code_circuit()
    elif code_name == 'steane':
        qc_circuit = steane_code_circuit()
    elif code_name == 'surface':
        qc_circuit = surface_code_circuit()
    else:
        raise ValueError(f"Unknown code name: {code_name}")
    
    # Transpile the circuit for the backend
    qc_transpiled = transpile(qc_circuit, backend, optimization_level=0)
    
    try:
        # Run the circuit on the backend with noise
        job = backend.run(qc_transpiled, noise_model=noise_model, shots=shots)
        result_noisy = job.result()
        noisy_counts = result_noisy.get_counts(qc_transpiled)
        
        # Define the ideal output as the all-zero bitstring
        ideal_counts = {'0' * qc_circuit.num_qubits: shots}

        # Calculate error rate based on counts
        error_rate = calculate_error_rate(ideal_counts, noisy_counts, shots)


        # Handling pure states: Converting to statevector and calculating fidelity
        qc_transpiled_no_measure = qc_circuit.remove_final_measurements(inplace=False)
        ideal_state = Statevector.from_label('0' * qc_circuit.num_qubits)
        noisy_state = Statevector.from_instruction(qc_transpiled_no_measure)
        fidelity = state_fidelity(noisy_state, ideal_state)
        
        return fidelity, error_rate
    except Exception as e:
        print(f"Error in simulation for {code_name} code: {str(e)}")
        return None, None

def simulate_with_all_noise_models(noise_level, backend):
    noise_models = {
        "Depolarizing": create_depolarizing_noise_model(noise_level),
        "Amplitude Damping": create_amplitude_damping_noise_model(noise_level),
        "Phase Damping": create_phase_damping_noise_model(noise_level),
        "Bit Flip": create_bit_flip_noise_model(noise_level),
        "Phase Flip": create_phase_flip_noise_model(noise_level),
        "Bit Phase Flip": create_bit_phase_flip_noise_model(noise_level),
        "Kraus": create_kraus_error_model(noise_level)
    }
    
    codes = ["shor", "steane", "surface"]
    results = []
    
    for code_name in codes:
        for noise_name, noise_model in noise_models.items():
            fidelity, error_rate = simulate_with_qec(noise_model, backend, code_name)
            results.append({
                "Code": code_name,
                "Noise Model": noise_name,
                "Fidelity": fidelity,
                "Error Rate": error_rate
            })
            print(f"Code: {code_name}, Noise Model: {noise_name}, Fidelity: {fidelity}, Error Rate: {error_rate}")
    
    return results

# Example: Simulate all codes with all noise models
noise_level = 0.01  # You can adjust this level as needed
results = simulate_with_all_noise_models(noise_level, backend)


/var/folders/f2/97psbk3s4nq1wknv9swc91740000gn/T/ipykernel_43956/790249972.py:23: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/api/migration-guides/qiskit-runtime.
  job = backend.run(qc_transpiled, noise_model=noise_model, shots=shots)


Code: shor, Noise Model: Depolarizing, Fidelity: 0.0, Error Rate: 0.9912109375
Code: shor, Noise Model: Amplitude Damping, Fidelity: 0.0, Error Rate: 0.9912109375
Code: shor, Noise Model: Phase Damping, Fidelity: 0.0, Error Rate: 0.9892578125
Code: shor, Noise Model: Bit Flip, Fidelity: 0.0, Error Rate: 0.9931640625
Code: shor, Noise Model: Phase Flip, Fidelity: 0.0, Error Rate: 0.9990234375
Code: shor, Noise Model: Bit Phase Flip, Fidelity: 0.0, Error Rate: 0.9912109375
Code: shor, Noise Model: Kraus, Fidelity: 0.0, Error Rate: 0.994140625
Code: steane, Noise Model: Depolarizing, Fidelity: 0.06249999999999996, Error Rate: 0.9599609375
Code: steane, Noise Model: Amplitude Damping, Fidelity: 0.06249999999999996, Error Rate: 0.9609375
Code: steane, Noise Model: Phase Damping, Fidelity: 0.06249999999999996, Error Rate: 0.966796875
Code: steane, Noise Model: Bit Flip, Fidelity: 0.06249999999999996, Error Rate: 0.970703125
Code: steane, Noise Model: Phase Flip, Fidelity: 0.06249999999999996

In [42]:
def simulate_with_all_noise_models(noise_level, backend):
    noise_models = {
        "Depolarizing": create_depolarizing_noise_model(noise_level),
        "Amplitude Damping": create_amplitude_damping_noise_model(noise_level),
        "Phase Damping": create_phase_damping_noise_model(noise_level),
        "Bit Flip": create_bit_flip_noise_model(noise_level),
        "Phase Flip": create_phase_flip_noise_model(noise_level),
        "Bit Phase Flip": create_bit_phase_flip_noise_model(noise_level),
        "Kraus": create_kraus_error_model(noise_level)
    }

In [ ]:
def simulate_no_correction(noise_model, backend):
    # Simple circuit without QEC
    qc = QuantumCircuit(1, 1)
    qc.h(0)
    
    # Transpile for the backend (without measurement for statevector)
    qc_transpiled_no_measure = transpile(qc, backend)
    
    # Evolve state vector without measurement
    state_noisy = Statevector.from_int(0, 2)
    state_noisy = state_noisy.evolve(qc_transpiled_no_measure)
    fidelity = state_fidelity(state_noisy, Statevector([1, 0]))  # Ideal state
    
    # Add measurement and transpile again for getting counts
    qc.measure(0, 0)
    qc_transpiled_with_measure = transpile(qc, backend)
    
    # Run the simulation with noise to get the error rate
    result = backend.run(qc_transpiled_with_measure, noise_model=noise_model, shots=1024).result()
    counts = result.get_counts()
    error_rate = 1 - (counts.get('0', 0) / 1024)
    
    return error_rate, fidelity

# Example usage:
noise_level = 0.05  # Adjust noise level as needed
# Assuming you have defined a noise model and backend properly:
# noise_model = ...
# backend = ...
error_rate, fidelity = simulate_no_correction(noise_model, backend)

print(f"Error Rate without QEC: {error_rate}")
print(f"Fidelity without QEC: {fidelity}")


/var/folders/f2/97psbk3s4nq1wknv9swc91740000gn/T/ipykernel_3327/1894121169.py:19: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/api/migration-guides/qiskit-runtime.
  result = backend.run(qc_transpiled_with_measure, noise_model=noise_model, shots=1024).result()


Error Rate without QEC: 0.505859375
Fidelity without QEC: 0.4999999999999999


In [53]:
from qiskit import QuantumCircuit, transpile, assemble, Aer, execute
from qiskit.quantum_info import Statevector, state_fidelity

def calculate_error_rate(ideal_counts, noisy_counts, shots):
    """Calculate the error rate based on measurement counts."""
    ideal_total = sum(ideal_counts.values())
    correct_counts = sum(noisy_counts.get(bitstring, 0) for bitstring in ideal_counts)
    error_rate = 1 - (correct_counts / shots)
    return error_rate

def get_circuit(code_name):
    """Returns the circuit for the specified error correction code."""
    if code_name == 'shor':
        return shor_code_circuit()
    elif code_name == 'steane':
        return steane_code_circuit()
    elif code_name == 'surface':
        return surface_code_circuit()
    else:
        raise ValueError(f"Unknown code name: {code_name}")

def simulate_with_qec_batch(backend, circuits, noise_models, shots=1024):
    """Simulate all circuits with different noise models in one batch job."""
    transpiled_circuits = [transpile(circuit, backend, optimization_level=0) for circuit in circuits]
    job = backend.run(transpiled_circuits, noise_model=noise_models, shots=shots)
    results = job.result()
    
    fidelities = []
    error_rates = []
    
    for circuit, transpiled_circuit, noise_model in zip(circuits, transpiled_circuits, noise_models):
        noisy_counts = results.get_counts(transpiled_circuit)
        ideal_counts = {'0' * circuit.num_qubits: shots}
        error_rate = calculate_error_rate(ideal_counts, noisy_counts, shots)
        
        qc_no_measure = circuit.remove_final_measurements(inplace=False)
        ideal_state = Statevector.from_label('0' * circuit.num_qubits)
        noisy_state = Statevector.from_instruction(qc_no_measure)
        fidelity = state_fidelity(noisy_state, ideal_state)
        
        fidelities.append(fidelity)
        error_rates.append(error_rate)
    
    return fidelities, error_rates

def test_with_gradual_noise(backend, code_names, initial_noise_level=0.001, step=0.001, max_noise_level=0.01):
    """Test with gradual noise levels for different error correction codes in one batch."""
    noise_level = initial_noise_level
    circuits = []
    noise_models = []
    metadata = []  # To keep track of the code, noise model, and noise level
    
    while noise_level <= max_noise_level:
        noise_models_dict = {
            "Depolarizing": create_depolarizing_noise_model(noise_level),
            "Amplitude Damping": create_amplitude_damping_noise_model(noise_level),
            "Phase Damping": create_phase_damping_noise_model(noise_level),
            "Bit Flip": create_bit_flip_noise_model(noise_level),
            "Phase Flip": create_phase_flip_noise_model(noise_level),
            "Bit Phase Flip": create_bit_phase_flip_noise_model(noise_level),
            "Kraus": create_kraus_error_model(noise_level)
        }
        
        for code_name in code_names:
            qc_circuit = get_circuit(code_name)
            for noise_name, noise_model in noise_models_dict.items():
                circuits.append(qc_circuit)
                noise_models.append(noise_model)
                metadata.append((noise_level, code_name, noise_name))
        
        noise_level += step
    
    fidelities, error_rates = simulate_with_qec_batch(backend, circuits, noise_models)
    
    # Display results
    for (noise_level, code_name, noise_name), fidelity, error_rate in zip(metadata, fidelities, error_rates):
        print(f"Noise Level: {noise_level}, Code: {code_name}, Noise Model: {noise_name}, Fidelity: {fidelity}, Error Rate: {error_rate}")

# Run tests for multiple codes in a single batch
test_with_gradual_noise(backend, ['shor', 'steane', 'surface'])


/var/folders/f2/97psbk3s4nq1wknv9swc91740000gn/T/ipykernel_3327/1337600240.py:23: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/api/migration-guides/qiskit-runtime.
  job = backend.run(qc_transpiled, noise_model=noise_model, shots=shots)


KeyboardInterrupt: 

In [ ]:
def analyze_circuit_depth(code_name):
    if code_name == 'shor':
        qc_circuit = shor_code_circuit(include_measure=False)
    elif code_name == 'steane':
        qc_circuit = steane_code_circuit(include_measure=False)
    elif code_name == 'surface':
        qc_circuit = surface_code_circuit(include_measure=False)
    else:
        raise ValueError(f"Unknown code name: {code_name}")

    # Transpile and analyze depth with explicit parameters
    qc_transpiled = transpile(
        qc_circuit, 
        backend, 
        optimization_level=0,
        coupling_map=backend.configuration().coupling_map,
        basis_gates=backend.configuration().basis_gates
    )
    depth = qc_transpiled.depth()
    print(f"Circuit depth for {code_name} code: {depth}")

# Analyze circuit depth
analyze_circuit_depth('shor')
analyze_circuit_depth('steane')
analyze_circuit_depth('surface')


Circuit depth for shor code: 225
Circuit depth for steane code: 103
Circuit depth for surface code: 274


In [ ]:


def simulate_code_threshold(noise_levels, code_name):
    thresholds = []
    fidelities = []
    results = []


    for noise_level in noise_levels:
        noise_models = {
            "Depolarizing": create_depolarizing_noise_model(noise_level),
            "Amplitude Damping": create_amplitude_damping_noise_model(noise_level),
            "Phase Damping": create_phase_damping_noise_model(noise_level),
            "Bit Flip": create_bit_flip_noise_model(noise_level),
            "Phase Flip": create_phase_flip_noise_model(noise_level),
            "Bit Phase Flip": create_bit_phase_flip_noise_model(noise_level),
            "Kraus": create_kraus_error_model(noise_level)
        }

        for noise_name, noise_model in noise_models.items():
            try:
                if code_name == 'shor':
                    qc_circuit = shor_code_circuit(include_measure=True)
                elif code_name == 'steane':
                    qc_circuit = steane_code_circuit(include_measure=True)
                elif code_name == 'surface':
                    qc_circuit = surface_code_circuit(include_measure=True)
                else:
                    raise ValueError(f"Unknown code name: {code_name}")

                qc_transpiled = transpile(qc_circuit, backend)
                job_noisy = backend.run(qc_transpiled, noise_model=noise_model, shots=1024)
                result_noisy = job_noisy.result(timeout=600)

                counts_noisy = result_noisy.get_counts(qc_transpiled)
                if counts_noisy is None:
                    continue

                ideal_counts = {'0' * qc_circuit.num_clbits: 1024}
                correct_counts = sum(counts_noisy.get(bitstring, 0) for bitstring in ideal_counts)
                error_rate = 1 - (correct_counts / 1024)

                # Handling pure states: Converting to statevector and calculating fidelity
                qc_transpiled_no_measure = qc_circuit.remove_final_measurements(inplace=False)
                ideal_state = Statevector.from_label('0' * qc_circuit.num_qubits)
                noisy_state = Statevector.from_instruction(qc_transpiled_no_measure)
                fidelity = state_fidelity(noisy_state, ideal_state)

                thresholds.append(error_rate)
                fidelities.append(fidelity)
                results.append({
                    'Code': code_name,
                    'Noise_Model': noise_name,
                    'Noise_Level': noise_level,
                    'Error_Rate': error_rate,
                    'Fidelity': fidelity
                })

                print(f"Simulation complete for {code_name} with {noise_name} at noise level {noise_level}:")
                print(f"Error Rate: {error_rate}")
                print(f"Fidelity: {fidelity}")

                time.sleep(10)

            except Exception as e:
                print(f"Error in simulation for {code_name} with {noise_name}: {str(e)}")
                continue

    return thresholds, fidelities, results

# Example usage:
noise_levels = [0.01, 0.02, 0.03]
shor_thresholds, shor_fidelities, shor_results = simulate_code_threshold(noise_levels, 'shor')
steane_thresholds, steane_fidelities, steane_results = simulate_code_threshold(noise_levels, 'steane')
surface_thresholds, surface_fidelities, surface_results = simulate_code_threshold(noise_levels, 'surface')


Simulation complete for shor with Depolarizing at noise level 0.01:
Error Rate: 0.998046875
Fidelity: 0.0
Simulation complete for shor with Amplitude Damping at noise level 0.01:
Error Rate: 1.0
Fidelity: 0.0
Simulation complete for shor with Phase Damping at noise level 0.01:
Error Rate: 1.0
Fidelity: 0.0
Simulation complete for shor with Bit Flip at noise level 0.01:
Error Rate: 1.0
Fidelity: 0.0
Simulation complete for shor with Phase Flip at noise level 0.01:
Error Rate: 1.0
Fidelity: 0.0
Simulation complete for shor with Bit Phase Flip at noise level 0.01:
Error Rate: 1.0
Fidelity: 0.0
Simulation complete for shor with Kraus at noise level 0.01:
Error Rate: 1.0
Fidelity: 0.0
Simulation complete for shor with Depolarizing at noise level 0.02:
Error Rate: 0.9990234375
Fidelity: 0.0
Simulation complete for shor with Amplitude Damping at noise level 0.02:
Error Rate: 1.0
Fidelity: 0.0
Simulation complete for shor with Phase Damping at noise level 0.02:
Error Rate: 1.0
Fidelity: 0.0
Si

In [ ]:
# Print the shapes and contents of the arrays
print("shor_thresholds shape:", shor_thresholds.shape, "content:", shor_thresholds)
print("steane_thresholds shape:", steane_thresholds.shape, "content:", steane_thresholds)
print("surface_thresholds shape:", surface_thresholds.shape, "content:", surface_thresholds)

print("shor_fidelities shape:", shor_fidelities.shape, "content:", shor_fidelities)
print("steane_fidelities shape:", steane_fidelities.shape, "content:", steane_fidelities)
print("surface_fidelities shape:", surface_fidelities.shape, "content:", surface_fidelities)


shor_thresholds shape: (21,) content: [0.99804688 1.         1.         1.         1.         1.
 1.         0.99902344 1.         1.         1.         1.
 1.         1.         0.99902344 1.         1.         1.
 1.         1.         1.        ]
steane_thresholds shape: (21,) content: [0.93457031 0.92480469 0.93359375 0.92871094 0.9453125  0.92382812
 0.94238281 0.9375     0.93652344 0.93066406 0.94335938 0.95117188
 0.94628906 0.93261719 0.94921875 0.921875   0.93261719 0.93847656
 0.93652344 0.94140625 0.93457031]
surface_thresholds shape: (21,) content: [0.88085938 0.88671875 0.88183594 0.87597656 0.85839844 0.86621094
 0.88964844 0.92773438 0.87792969 0.89257812 0.88183594 0.88085938
 0.86621094 0.87304688 0.9453125  0.88085938 0.86621094 0.86914062
 0.85351562 0.88574219 0.87597656]
shor_fidelities shape: (21,) content: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
steane_fidelities shape: (21,) content: [0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.06

In [ ]:

print("Surface code simulation result:", surface_result)
surface_thresholds.append(surface_error_rate)
surface_fidelities.append(surface_fidelity)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Convert lists to numpy arrays
shor_thresholds = np.array(shor_thresholds)
steane_thresholds = np.array(steane_thresholds)
surface_thresholds = np.array(surface_thresholds)

shor_fidelities = np.array(shor_fidelities)
steane_fidelities = np.array(steane_fidelities)
surface_fidelities = np.array(surface_fidelities)

# Check the dimensions and compute means only if valid
if shor_thresholds.ndim == 2 and shor_thresholds.shape[1] == len(noise_levels):
    shor_mean_thresholds = np.mean(shor_thresholds, axis=0)
else:
    print("Issue with shor_thresholds dimensions")
    shor_mean_thresholds = np.array([])

if steane_thresholds.ndim == 2 and steane_thresholds.shape[1] == len(noise_levels):
    steane_mean_thresholds = np.mean(steane_thresholds, axis=0)
else:
    print("Issue with steane_thresholds dimensions")
    steane_mean_thresholds = np.array([])

if surface_thresholds.ndim == 2 and surface_thresholds.shape[1] == len(noise_levels):
    surface_mean_thresholds = np.mean(surface_thresholds, axis=0)
else:
    print("Issue with surface_thresholds dimensions")
    surface_mean_thresholds = np.array([])

if shor_fidelities.ndim == 2 and shor_fidelities.shape[1] == len(noise_levels):
    shor_mean_fidelities = np.mean(shor_fidelities, axis=0)
else:
    print("Issue with shor_fidelities dimensions")
    shor_mean_fidelities = np.array([])

if steane_fidelities.ndim == 2 and steane_fidelities.shape[1] == len(noise_levels):
    steane_mean_fidelities = np.mean(steane_fidelities, axis=0)
else:
    print("Issue with steane_fidelities dimensions")
    steane_mean_fidelities = np.array([])

if surface_fidelities.ndim == 2 and surface_fidelities.shape[1] == len(noise_levels):
    surface_mean_fidelities = np.mean(surface_fidelities, axis=0)
else:
    print("Issue with surface_fidelities dimensions")
    surface_mean_fidelities = np.array([])

# Ensure the arrays are not empty before plotting
if len(shor_mean_thresholds) > 0:
    plt.figure(figsize=(10, 6))
    plt.plot(noise_levels, shor_mean_thresholds, label="Shor Code Mean Error Rate", marker='o')
    plt.plot(noise_levels, steane_mean_thresholds, label="Steane Code Mean Error Rate", marker='o')
    plt.plot(noise_levels, surface_mean_thresholds, label="Surface Code Mean Error Rate", marker='o')
    plt.xlabel("Noise Probability")
    plt.ylabel("Mean Error Rate")
    plt.legend()
    plt.title("Mean Error Rate Comparison for Shor, Steane, and Surface Codes")
    plt.grid(True)
    plt.show()

if len(shor_mean_fidelities) > 0:
    plt.figure(figsize=(10, 6))
    plt.plot(noise_levels, shor_mean_fidelities, label="Shor Code Mean Fidelity", marker='o')
    plt.plot(noise_levels, steane_mean_fidelities, label="Steane Code Mean Fidelity", marker='o')
    plt.plot(noise_levels, surface_mean_fidelities, label="Surface Code Mean Fidelity", marker='o')
    plt.xlabel("Noise Probability")
    plt.ylabel("Mean Fidelity")
    plt.legend()
    plt.title("Mean Fidelity Comparison for Shor, Steane, and Surface Codes")
    plt.grid(True)
    plt.show()


Issue with shor_thresholds dimensions
Issue with steane_thresholds dimensions
Issue with surface_thresholds dimensions
Issue with shor_fidelities dimensions
Issue with steane_fidelities dimensions
Issue with surface_fidelities dimensions


In [ ]:

def analyze_resources(backend, code_name):
    if code_name == 'shor':
        qc_circuit = shor_code_circuit(include_measure=True)
    elif code_name == 'steane':
        qc_circuit = steane_code_circuit(include_measure=True)
    elif code_name == 'surface':
        qc_circuit = surface_code_circuit(include_measure=True)
    else:
        raise ValueError(f"Unknown code name: {code_name}")
    
    # Use backend.target for transpilation
    qc_transpiled = transpile(
        qc_circuit,
        backend,
        target=backend.target
    )
    
    # Calculate resources
    depth = qc_transpiled.depth()
    gate_counts = qc_transpiled.count_ops()
    qubit_count = qc_transpiled.num_qubits
    
    # Calculate qubit overhead (physical qubits needed minus logical qubit)
    logical_qubit_count = 1  # Assuming we're encoding one logical qubit in these codes
    qubit_overhead = qubit_count - logical_qubit_count
    
    print(f"Code: {code_name}")
    print(f"Circuit Depth: {depth}")
    print(f"Gate Counts: {gate_counts}")
    print(f"Total Qubit Count: {qubit_count}")
    print(f"Qubit Overhead: {qubit_overhead}")
    
    return {
        'Code': code_name,
        'Circuit Depth': depth,
        'Gate Counts': gate_counts,
        'Total Qubit Count': qubit_count,
        'Qubit Overhead': qubit_overhead
    }

# Analyze resources and qubit overhead for each code

shor_resources = analyze_resources(backend, 'shor')
steane_resources = analyze_resources(backend, 'steane')
surface_resources = analyze_resources(backend, 'surface')

# Save resource usage results including qubit overhead
import csv
with open('code_resource_usage.csv', 'w', newline='') as csvfile:
    fieldnames = ['Code', 'Circuit Depth', 'Gate Counts', 'Total Qubit Count', 'Qubit Overhead']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    writer.writerow(shor_resources)
    writer.writerow(steane_resources)
    writer.writerow(surface_resources)


Code: shor
Circuit Depth: 72
Gate Counts: OrderedDict({'rz': 114, 'sx': 82, 'ecr': 45, 'x': 11, 'measure': 9, 'barrier': 1})
Total Qubit Count: 127
Qubit Overhead: 126
Code: steane
Circuit Depth: 39
Gate Counts: OrderedDict({'rz': 49, 'sx': 30, 'ecr': 15, 'measure': 7, 'barrier': 1})
Total Qubit Count: 127
Qubit Overhead: 126
Code: surface
Circuit Depth: 119
Gate Counts: OrderedDict({'rz': 96, 'sx': 72, 'ecr': 39, 'measure': 13, 'x': 9, 'barrier': 1})
Total Qubit Count: 127
Qubit Overhead: 126


In [ ]:

from qiskit.result import Counts

def simulate_fault_tolerance(noise_level, backend, codes, repetitions=10):
    noise_models = {
        'Depolarizing': create_depolarizing_noise_model(noise_level),
        'Amplitude Damping': create_amplitude_damping_noise_model(noise_level),
        'Phase Damping': create_phase_damping_noise_model(noise_level),
        'Bit Flip': create_bit_flip_noise_model(noise_level),
        'Phase Flip': create_phase_flip_noise_model(noise_level),
        'Bit Phase Flip': create_bit_phase_flip_noise_model(noise_level),
        'Kraus': create_kraus_error_model(noise_level)
    }
    
    all_results = {}
    
    for code_name in codes:
        results = []
        
        for noise_model_name, noise_model in noise_models.items():
            for i in range(repetitions):
                if code_name == 'shor':
                    qc_circuit = shor_code_circuit(include_measure=True)
                elif code_name == 'steane':
                    qc_circuit = steane_code_circuit(include_measure=True)
                elif code_name == 'surface':
                    qc_circuit = surface_code_circuit(include_measure=True)
                else:
                    raise ValueError(f"Unknown code name: {code_name}")

                # Transpile the circuit with minimal optimization
                qc_transpiled = transpile(qc_circuit, backend, optimization_level=0)

                try:
                    # Run the noisy circuit
                    job = backend.run(qc_transpiled, noise_model=noise_model, shots=1024)
                    result = job.result()
                    counts_noisy = result.get_counts()

                    # Calculate error rate
                    correct_counts = counts_noisy.get('0' * qc_transpiled.num_clbits, 0)
                    error_rate = 1 - (correct_counts / 1024)

                    # You can use fidelity of measurement outcomes as a proxy if needed
                    # Ideally, you would compare distributions here if required
                    
                    results.append({
                        'Code': code_name,
                        'Noise Model': noise_model_name,
                        'Error Rate': error_rate,
                        'Repetition': i + 1
                    })

                    print(f"Code: {code_name}, Noise Model: {noise_model_name}, Error Rate: {error_rate}, Repetition: {i + 1}")

                except Exception as e:
                    print(f"Error in processing: {e}")
                    continue
        
        all_results[code_name] = results
    
    return all_results

# Usage
codes = ['shor', 'steane', 'surface']

noise_level = 0.05
fault_tolerance_results = simulate_fault_tolerance(noise_level=noise_level, backend=backend, codes=codes, repetitions=10)

# Save results for each code to separate CSV files
import csv
for code_name, results in fault_tolerance_results.items():
    csv_filename = f'{code_name}_fault_tolerance_results.csv'
    with open(csv_filename, 'w', newline='') as csvfile:
        fieldnames = ['Repetition', 'Code', 'Noise Model', 'Error Rate']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for result in results:
            writer.writerow(result)

    print(f"Results for {code_name} saved to {csv_filename}")


/var/folders/f2/97psbk3s4nq1wknv9swc91740000gn/T/ipykernel_3327/1597464355.py:35: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/api/migration-guides/qiskit-runtime.
  job = backend.run(qc_transpiled, noise_model=noise_model, shots=1024)


Code: shor, Noise Model: Depolarizing, Error Rate: 0.9970703125, Repetition: 1
Code: shor, Noise Model: Depolarizing, Error Rate: 0.990234375, Repetition: 2
Code: shor, Noise Model: Depolarizing, Error Rate: 0.9794921875, Repetition: 3
Code: shor, Noise Model: Depolarizing, Error Rate: 0.9853515625, Repetition: 4
Error in processing: 'Error submitting job: "HTTPSConnectionPool(host=\'api.quantum.ibm.com\', port=443): Read timed out. (read timeout=900)"'
Code: shor, Noise Model: Depolarizing, Error Rate: 0.9931640625, Repetition: 6
Code: shor, Noise Model: Depolarizing, Error Rate: 0.99609375, Repetition: 7


KeyboardInterrupt: 

In [39]:
def simulate_with_all_noise_models(noise_level, backend):
    noise_models = {
        "Depolarizing": create_depolarizing_noise_model(noise_level),
        "Amplitude Damping": create_amplitude_damping_noise_model(noise_level),
        "Phase Damping": create_phase_damping_noise_model(noise_level),
        "Bit Flip": create_bit_flip_noise_model(noise_level),
        "Phase Flip": create_phase_flip_noise_model(noise_level),
        "Bit Phase Flip": create_bit_phase_flip_noise_model(noise_level),
        "Kraus": create_kraus_error_model(noise_level)
    }

    # Convert the noise_models dictionary to a list of dictionaries
    noise_model_list = []
    for model_name, noise_model in noise_models.items():
        noise_model_list.append({
            "Noise Model": model_name,
            #"Fidelity": calculate_fidelity(noise_model, backend),
            #"Error Rate": calculate_error_rate(noise_model, backend),
            "Code": "some_code"  # Replace "some_code" with the appropriate value
        })
    
    return noise_model_list


In [40]:



def evaluate_performance(backend, code_name):
    # Get noise models from your existing function
    noise_models = simulate_with_all_noise_models(0.05, backend)
    
    # Ensure noise_models is a list of dictionaries as returned by your simulate_with_all_noise_models function
    if not isinstance(noise_models, list):
        raise ValueError("The simulate_with_all_noise_models function should return a list.")

    if code_name == 'shor':
        qc_circuit = shor_code_circuit(include_measure=True)
    elif code_name == 'steane':
        qc_circuit = steane_code_circuit(include_measure=True)
    elif code_name == 'surface':
        qc_circuit = surface_code_circuit(include_measure=True)
    else:
        raise ValueError(f"Unknown code name: {code_name}")
    
    # Start memory and time profiling
    tracemalloc.start()
    start_time = time.time()
    
    qc_transpiled = transpile(qc_circuit, backend)
    
    performance_results = []
    
    for result in noise_models:
        if result['Code'] == code_name:
            noise_model_name = result['Noise Model']
            fidelity = result['Fidelity']
            error_rate = result['Error Rate']

            end_time = time.time()
            execution_time = end_time - start_time

            current, peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()

            print(f"Code: {code_name}, Noise Model: {noise_model_name}, Execution Time: {execution_time} seconds, Peak Memory Usage: {peak / 10**6} MB")
            
            performance_results.append({
                'Code': code_name,
                'Noise Model': noise_model_name,
                'Execution Time (Seconds)': execution_time,
                'Peak Memory Usage (MB)': peak / 10**6
            })
    
    return performance_results

# Performance analysis for Shor, Steane, and Surface codes on the current environment
for code_name in ['shor', 'steane', 'surface']:
    performance = evaluate_performance(backend, code_name)

    # Save performance results to a CSV file
    csv_filename = f'{code_name}_performance_comparison.csv'
    with open(csv_filename, 'w', newline='') as csvfile:
        fieldnames = ['Code', 'Noise Model', 'Execution Time (Seconds)', 'Peak Memory Usage (MB)']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for result in performance:
            writer.writerow(result)

    print(f"Results for {code_name} saved to {csv_filename}")


Results for shor saved to shor_performance_comparison.csv
Results for steane saved to steane_performance_comparison.csv
Results for surface saved to surface_performance_comparison.csv


In [90]:
{
    "Depolarizing": create_depolarizing_noise_model,
    "Amplitude Damping": create_amplitude_damping_noise_model,
    # Add other noise models here
}


{'Depolarizing': <function __main__.create_depolarizing_noise_model(noise_level)>,
 'Amplitude Damping': <function __main__.create_amplitude_damping_noise_model(noise_level)>}

In [41]:
def analyze_resources(backend, code_name):
    if code_name == 'shor':
        qc_circuit = shor_code_circuit(include_measure=True)
    elif code_name == 'steane':
        qc_circuit = steane_code_circuit(include_measure=True)
    elif code_name == 'surface':
        qc_circuit = surface_code_circuit(include_measure=True)
    else:
        raise ValueError(f"Unknown code name: {code_name}")
    
    # Transpile the circuit for the backend
    qc_transpiled = transpile(qc_circuit, backend)
    
    # Calculate resources
    depth = qc_transpiled.depth()
    gate_counts = qc_transpiled.count_ops()
    qubit_count = qc_transpiled.num_qubits
    
    # Calculate qubit overhead (physical qubits needed minus logical qubit)
    logical_qubit_count = 1  # Assuming we're encoding one logical qubit in these codes
    qubit_overhead = qubit_count - logical_qubit_count
    
    print(f"Code: {code_name}")
    print(f"Circuit Depth: {depth}")
    print(f"Gate Counts: {gate_counts}")
    print(f"Total Qubit Count: {qubit_count}")
    print(f"Qubit Overhead: {qubit_overhead}")
    
    return {
        'Code': code_name,
        'Circuit Depth': depth,
        'Gate Counts': gate_counts,
        'Total Qubit Count': qubit_count,
        'Qubit Overhead': qubit_overhead
    }

# Analyze resources and qubit overhead for each code
for code_name in ['shor', 'steane', 'surface']:
    resources = analyze_resources(backend, code_name)
    
    # Save resource usage results to a CSV file
    csv_filename = f'{code_name}_resource_usage.csv'
    with open(csv_filename, 'w', newline='') as csvfile:
        fieldnames = ['Code', 'Circuit Depth', 'Gate Counts', 'Total Qubit Count', 'Qubit Overhead']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        writer.writerow(resources)

    print(f"Resource usage for {code_name} saved to {csv_filename}")


Code: shor
Circuit Depth: 77
Gate Counts: OrderedDict({'rz': 121, 'sx': 83, 'ecr': 45, 'measure': 9, 'x': 8, 'barrier': 1})
Total Qubit Count: 127
Qubit Overhead: 126
Resource usage for shor saved to shor_resource_usage.csv
Code: steane
Circuit Depth: 50
Gate Counts: OrderedDict({'rz': 59, 'sx': 38, 'ecr': 18, 'measure': 7, 'x': 1, 'barrier': 1})
Total Qubit Count: 127
Qubit Overhead: 126
Resource usage for steane saved to steane_resource_usage.csv
Code: surface
Circuit Depth: 123
Gate Counts: OrderedDict({'rz': 99, 'sx': 74, 'ecr': 39, 'measure': 13, 'x': 9, 'barrier': 1})
Total Qubit Count: 127
Qubit Overhead: 126
Resource usage for surface saved to surface_resource_usage.csv
